In [1]:
pip install snowflake-connector-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.7/74.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.4/479.4 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: cffi
    Found existing installation: cffi 2.0.0
    Uninstalling cffi-2.0.0:
      Successfully uninstalled cffi-2.0.0


In [13]:
import getpass
pwd = getpass.getpass("Enter Password")


Enter Password··········


In [14]:
import snowflake.connector
import pandas as pd
import json
import requests


In [15]:
print(pwd)

Snowflake092025


In [16]:
conn = snowflake.connector.connect(
    user = 'sanyam2025',
    password = str(pwd),
    account = 'FBTWOMY-YM58029',
    warehouse = 'COMPUTE_WH',
    database = 'CLINICAL_DB',
    schema = 'RAW'
)

In [17]:
curr = conn.cursor()
RAW_TABLE = "raw_ctgov__studies"
curr.execute(f"create table if not exists {RAW_TABLE} \
(raw_json variant, load_ts timestamp default current_timestamp())")

###Validating the ClinicalTrials.gov API data

In [30]:
api_url = "https://clinicaltrials.gov/api/v2/studies"
param = {
        "pageSize": 10
    }
query_params = {
        "pageSize": 1000
    }
raw_table = "raw_ctgov__studies"

In [25]:
# validating data:

response = requests.get(api_url, params=param)
response.raise_for_status()
data = response.json()
print(json.dumps(data, indent=2))

{
  "studies": [
    {
      "protocolSection": {
        "identificationModule": {
          "nctId": "NCT05840835",
          "orgStudyIdInfo": {
            "id": "IMX-110-002"
          },
          "organization": {
            "fullName": "Immix Biopharma, Inc.",
            "class": "INDUSTRY"
          },
          "briefTitle": "Study of IMX-110 in Combination With Tislelizumab in Patients With Advanced Solid Tumors",
          "officialTitle": "A Phase 1/2a Open-Label, Dose-Escalation/Dose-Expansion Safety, Tolerability and Anti-tumor Activity Study of IMX-110 in Combination With Tislelizumab in Patients With Advanced Solid Tumors",
          "acronym": "IMMINENT-01"
        },
        "statusModule": {
          "statusVerifiedDate": "2023-08",
          "overallStatus": "UNKNOWN",
          "lastKnownStatus": "RECRUITING",
          "expandedAccessInfo": {
            "hasExpandedAccess": false
          },
          "startDateStruct": {
            "date": "2023-02-03",
  

In [26]:
print(data.keys())
print(data['nextPageToken'])
print(data['studies'])
print(data['studies'][1])

dict_keys(['studies', 'nextPageToken'])
ZVJj7o2Eke8v3lpjRM2o5br2mo6QcZFrbPKg0A
[{'protocolSection': {'identificationModule': {'nctId': 'NCT05840835', 'orgStudyIdInfo': {'id': 'IMX-110-002'}, 'organization': {'fullName': 'Immix Biopharma, Inc.', 'class': 'INDUSTRY'}, 'briefTitle': 'Study of IMX-110 in Combination With Tislelizumab in Patients With Advanced Solid Tumors', 'officialTitle': 'A Phase 1/2a Open-Label, Dose-Escalation/Dose-Expansion Safety, Tolerability and Anti-tumor Activity Study of IMX-110 in Combination With Tislelizumab in Patients With Advanced Solid Tumors', 'acronym': 'IMMINENT-01'}, 'statusModule': {'statusVerifiedDate': '2023-08', 'overallStatus': 'UNKNOWN', 'lastKnownStatus': 'RECRUITING', 'expandedAccessInfo': {'hasExpandedAccess': False}, 'startDateStruct': {'date': '2023-02-03', 'type': 'ACTUAL'}, 'primaryCompletionDateStruct': {'date': '2024-01-24', 'type': 'ESTIMATED'}, 'completionDateStruct': {'date': '2024-01-24', 'type': 'ESTIMATED'}, 'studyFirstSubmitDate

In [27]:
val = data.get('studies', [])
df = pd.json_normalize(val)
print(df.columns)

Index(['hasResults', 'protocolSection.identificationModule.nctId',
       'protocolSection.identificationModule.orgStudyIdInfo.id',
       'protocolSection.identificationModule.organization.fullName',
       'protocolSection.identificationModule.organization.class',
       'protocolSection.identificationModule.briefTitle',
       'protocolSection.identificationModule.officialTitle',
       'protocolSection.identificationModule.acronym',
       'protocolSection.statusModule.statusVerifiedDate',
       'protocolSection.statusModule.overallStatus',
       'protocolSection.statusModule.lastKnownStatus',
       'protocolSection.statusModule.expandedAccessInfo.hasExpandedAccess',
       'protocolSection.statusModule.startDateStruct.date',
       'protocolSection.statusModule.startDateStruct.type',
       'protocolSection.statusModule.primaryCompletionDateStruct.date',
       'protocolSection.statusModule.primaryCompletionDateStruct.type',
       'protocolSection.statusModule.completionDateSt

##Fetching the ClinicalTrials.gov API data

In [29]:
import snowflake.connector
import pandas as pd
import json
import requests

api_url = "https://clinicaltrials.gov/api/v2/studies"

def get_data(api):
    next_page_token = None

    query_params = {
        "pageSize": 1000
    }

    while True:
        if next_page_token:
            query_params['pageToken'] = next_page_token

        response = requests.get(api_url, params=query_params)
        response.raise_for_status() # Exit if an HTTP error occurs
        data = response.json()

        studies = data.get('studies', [])
        yield from studies # Yield studies as they are fetched

        next_page_token = data.get('nextPageToken')
        if not next_page_token:
            break

    print("API fetch complete.")

In [22]:
get_data(api_url)

Fetched 1000 studies. Total so far: 1000
Fetched 1000 studies. Total so far: 2000
Fetched 1000 studies. Total so far: 3000
Fetched 1000 studies. Total so far: 4000
Fetched 1000 studies. Total so far: 5000
Fetched 1000 studies. Total so far: 6000
Fetched 1000 studies. Total so far: 7000
Fetched 1000 studies. Total so far: 8000
Fetched 1000 studies. Total so far: 9000
Fetched 1000 studies. Total so far: 10000
Fetched 1000 studies. Total so far: 11000
Fetched 1000 studies. Total so far: 12000
Fetched 1000 studies. Total so far: 13000
Fetched 1000 studies. Total so far: 14000
Fetched 1000 studies. Total so far: 15000
Fetched 1000 studies. Total so far: 16000
Fetched 1000 studies. Total so far: 17000
Fetched 1000 studies. Total so far: 18000
Fetched 1000 studies. Total so far: 19000
Fetched 1000 studies. Total so far: 20000
Fetched 1000 studies. Total so far: 21000
Fetched 1000 studies. Total so far: 22000
Fetched 1000 studies. Total so far: 23000
Fetched 1000 studies. Total so far: 24000
F

KeyboardInterrupt: 

In [31]:
curr = conn.cursor()
curr.execute(f"create table if not exists {raw_table} \
(raw_json variant, load_ts timestamp default current_timestamp())")

## Loading data to snowflake

In [39]:
# Function to load the api data into snowflake table

def load_data(studies_generator):
  insert_sql = f"insert into {RAW_TABLE} (raw_json) select parse_json(%s)" # Added parse_json
  for study in studies_generator:
    curr.execute(insert_sql, (json.dumps(study),))
    print(f"Inserted one study.")

  print(f"Data inserted to table: {RAW_TABLE} successfully")

In [ ]:
# Function call

api_data_generator = get_data(api_url) # get_data is now a generator
load_data(api_data_generator)

Streaming output truncated to the last 5000 lines.
Inserted one study.
Inserted one study.
Inserted one study.
Inserted one study.
Inserted one study.
Inserted one study.
Inserted one study.
Inserted one study.
Inserted one study.
Inserted one study.
Inserted one study.
Inserted one study.
Inserted one study.
Inserted one study.
Inserted one study.
Inserted one study.
Inserted one study.
Inserted one study.
Inserted one study.
Inserted one study.
Inserted one study.
Inserted one study.
Inserted one study.
Inserted one study.
Inserted one study.
Inserted one study.
Inserted one study.
Inserted one study.
Inserted one study.
Inserted one study.
Inserted one study.
Inserted one study.
Inserted one study.
Inserted one study.
Inserted one study.
Inserted one study.
Inserted one study.
Inserted one study.
Inserted one study.
Inserted one study.
Inserted one study.
Inserted one study.
Inserted one study.
Inserted one study.
Inserted one study.
Inserted one study.
Inserted one study.
Inserted 